In [1]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv('output.csv')

In [3]:
# Split the 'Game' column into separate columns
df[['Sport', 'Event Details', 'Date Time']] = raw_df['Game'].str.split('\n+', expand=True)

# Drop the original 'Game' column if no longer needed
df.drop('Game', axis=1, inplace=True)

# Strip leading and trailing whitespaces from the new columns
#df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

# Split 'Date Time' column into 'Date' and 'Time'
df[['Date', 'Time']] = df['Date Time'].str.split('|', expand=True)
df.drop('Date Time', axis=1, inplace=True)

# Split 'Sport' column into 'Main Sport' and 'Sub Sport'
df[['Main Sport', 'Sub Sport']] = df['Sport'].str.split('. ', n=1, expand=True)
df.drop('Sport', axis=1, inplace=True)

# Split 'Event Details' column into 'Home' and 'Away'
df[['Home', 'Away']] = df['Event Details'].str.split('-', n=1, expand=True)
df.drop('Event Details', axis=1, inplace=True)

In [ ]:
#reorder columns to 'Time' 'Bet Type',  'Main Sport', 'Sub Sport', 'Home', 'Away' 'Odds', 'Status'
df = df[['Time', 'Bet Type', 'Main Sport', 'Sub Sport', 'Home', 'Away', 'Odds', 'Status']]
df.head(10)

In [16]:
#save the data to csv
df.to_csv('1xbet_history.csv')

In [2]:
df = pd.read_csv('1xbet_history.csv', parse_dates=['Date'], dayfirst=True)

### most profitable sport

In [ ]:
# Get the number of wins per sport type
total_counts = df.groupby('Main Sport').size()

# Filter the DataFrame for wins and count per sport type
win_counts = df[df['Status'] == 'win'].groupby('Main Sport').size()

# Create a new DataFrame with win counts and total counts
win_stats = pd.DataFrame({
    'Win Count': win_counts,
    'Total Count': total_counts
})

# Calculate win percentage and add it as a new column in the DataFrame
win_stats['Win Percentage'] = (win_stats['Win Count'] / win_stats['Total Count']) * 100
#win_stats.sort_values('Total Count', ascending=False)

In [4]:
win_stats.sort_values(['Win Count','Total Count'], ascending=False)

,Win Count,Total Count,Win Percentage
Main Sport,,,
Football,474.0,916,51.746725
Basketball,115.0,237,48.523207
Volleyball,14.0,21,66.666667
Tennis,7.0,12,58.333333
Ic,3.0,9,33.333333
Tabl,2.0,3,66.666667
Baseball,2.0,2,100.000000
Cricket,1.0,1,100.000000
Rugby,1.0,1,100.000000


###   Calculates bet type counts, wins, and win ratios for a specific game type.

In [6]:
def get_bet_type_stats(df, game_type):
  """
  Calculates bet type counts, wins, and win ratios for a specific game type.

  Args:
    df: Pandas DataFrame containing betting data.
    game_type: String representing the desired game type (e.g., "Football", "Basketball").

  Returns:
    Pandas DataFrame with bet type counts, wins, and win ratios.
  """

  # Filter for the specified game type and exclude NaN values in 'Bet Type'
  df_filtered = df[df['Main Sport'] == game_type].dropna(subset=['Bet Type'])

  # Count the number of bets and wins per bet type
  bet_type_counts = df_filtered.groupby('Bet Type')['Status'].count()
  bet_type_wins = df_filtered[df_filtered['Status'] == 'win'].groupby('Bet Type')['Status'].count()

  # Calculate win ratio and combine with counts
  bet_type_win_ratio = (bet_type_wins / bet_type_counts) * 100

  # Rename columns and combine into DataFrame
  bet_type_stats = pd.concat([bet_type_counts, bet_type_wins, bet_type_win_ratio], axis=1, keys=['Bet Count', 'Win Count', 'Win Ratio (%)'])

  return bet_type_stats


In [ ]:
soccer = get_bet_type_stats(df.copy(), "Football")

In [14]:
soccer.sort_values(['Bet Count','Win Ratio (%)'], ascending=False).head(20)

,Bet Count,Win Count,Win Ratio (%)
Bet Type,,,
W1,227,124.0,54.625551
W2,138,56.0,40.579710
Total Over (2.5),87,52.0,59.770115
1X,58,45.0,77.586207
Both Teams To Score - Yes,37,16.0,43.243243
X,35,6.0,17.142857
Total Over (0.5),32,26.0,81.250000
2X,27,12.0,44.444444
European Handicap (0:1) W1,16,9.0,56.250000


In [15]:
def get_daily_bet_stats(df):
    """
    Calculates average number of bets placed per day and their win rate.

    Args:
        df: Pandas DataFrame containing betting data with a "Date" column.

    Returns:
        Pandas DataFrame with average bets and win rate, or a detailed DataFrame with daily stats.
    """

    #df['Date'] = pd.to_datetime(df['Date'])
    daily_bets = df.groupby('Date')['Status'].count()
    daily_wins = df[df['Status'] == 'win'].groupby('Date')['Status'].count()
    daily_win_rate = (daily_wins / daily_bets) * 100
    if df.empty:
        return None, None  # Or handle appropriately

    try:
        average_bets = daily_bets.mean(index=[0])  # Attempt with index
        average_win_rate = daily_win_rate.mean(index=[0])
    except TypeError:
        average_bets = daily_bets.mean()  # Fallback without index
        average_win_rate = daily_win_rate.mean()

    # Provide options for overall or detailed stats
    daily_stats_overall = pd.DataFrame({'Average Bets': average_bets, 'Average Win Rate (%)': average_win_rate}, index=[0])

    #daily_stats_overall = pd.DataFrame({'Average Bets': average_bets, 'Average Win Rate (%)': average_win_rate})
    daily_stats_detailed = pd.concat([daily_bets, daily_wins, daily_win_rate], axis=1, keys=['Bet Count', 'Win Count', 'Win Ratio (%)'])

    return daily_stats_overall, daily_stats_detailed

In [16]:
overall_stats, detailed_stats = get_daily_bet_stats(df.copy())

In [19]:
detailed_stats

,Bet Count,Win Count,Win Ratio (%)
Date,,,
2023-10-05,9,5.0,55.555556
2023-10-07,4,3.0,75.000000
2023-10-08,5,3.0,60.000000
2023-10-11,10,6.0,60.000000
2023-10-12,1,NaN,NaN
...,...,...,...
2023-12-29,33,11.0,33.333333
2023-12-30,13,4.0,30.769231
2023-12-31,12,7.0,58.333333


In [22]:
# For daily details
detailed_stats.sort_values('Bet Count', ascending=False).head(20)

,Bet Count,Win Count,Win Ratio (%)
Date,,,
2023-12-15,36,19.0,52.777778
2023-12-23,35,12.0,34.285714
2023-11-09,33,18.0,54.545455
2023-12-29,33,11.0,33.333333
2023-12-16,31,17.0,54.838710
2023-12-22,30,15.0,50.000000
2023-11-12,29,12.0,41.379310
2023-11-17,27,15.0,55.555556
2023-12-26,27,13.0,48.148148


In [18]:
  # For overall averages
overall_stats

,Average Bets,Average Win Rate (%)
0,15.227848,52.029898


In [5]:
df[df['Bet Type'] == 'Total Under (11.5)']

,Date,Time,Bet Type,Main Sport,Sub Sport,Home,Away,Odds,Status
909,2023-11-09,11:00,Total Under (11.5),Football,Indonesia. Liga 1,Corners PS Barito Putera,Corners Persebaya 1927,1.41,win
917,2023-11-08,23:00,Total Under (11.5),Football,UEFA Champions League,Corners Bayern Munich,Corners Galatasaray,1.47,win
918,2023-11-08,23:00,Total Under (11.5),Football,UEFA Champions League,Corners Copenhagen,Corners Manchester United,1.41,win
919,2023-11-08,23:00,Total Under (11.5),Football,UEFA Champions League,Corners Real Madrid,Corners Braga,1.56,win
950,2023-11-05,17:00,Total Under (11.5),Football,England. Premier League,Corners Nottingham Forest,Corners Aston Villa,1.46,win
952,2023-11-05,17:30,Total Under (11.5),Football,Germany. Bundesliga,Corners VfL Wolfsburg,Corners Werder Bremen,1.35,win
975,2023-11-04,13:30,Total Under (11.5),Football,Turkey. SuperLiga,Corners İstanbulspor,Corners Pendikspor,1.29,win
991,2023-11-03,11:45,Total Under (11.5),Football,Australia. A League,Corners Melbourne City,Corners Sydney,1.58,win
1080,2023-10-29,17:00,Total Under (11.5),Football,England. Premier League,Corners Liverpool,Corners Nottingham Forest,1.45,win
1081,2023-10-29,17:00,Total Under (11.5),Football,England. Premier League,Corners Brighton & Hove Albion,Corners Fulham,1.60,win


In [7]:
# If 'Odd' is not numeric, you should convert it first using pd.to_numeric
df['Odd'] = pd.to_numeric(df['Odds'], errors='coerce')

# Filter the DataFrame for wins
wins_df = df[df['Status'] == 'win']

# Group by 'Sport Type' and calculate the mean of 'Odd'
average_winning_odds = wins_df.groupby('Main Sport')['Odds'].mean()

print(average_winning_odds)

Main Sport
Baseball      1.575000
Basketball    1.961826
Cricket       1.330000
Football      1.787616
Ic            2.043333
Rugby         2.010000
Tabl          1.400000
Tennis        1.534286
Volleyball    1.730000
Name: Odds, dtype: float64


### optimal number of bets in a days


#optimal number of bets in a days
#types of bets with highest win ration per sport
#day with the highest number of losses
#optimum/average odd resulting onto a win 
#most profitable day
#time of day when most bets are placed
#time of day when bets placed are profitable/riskier

In [33]:
df_copy = df.set_index('Date')

In [39]:
daily_stats = df_copy.groupby(pd.Grouper(level='Date', freq='W-MON')).agg(
    bets=('Status', 'count'),
    wins=('Status', lambda x: x.eq('win').sum())  # Count "win" occurrences
)

In [ ]:
# Now perform the calculation
daily_stats['win_ratio'] = (daily_stats['wins'] / daily_stats['bets']) * 100
daily_stats

In [47]:
daily_stats

,bets,wins,win_ratio
Date,,,
2023-10-09,18,11,61.111111
2023-10-16,11,6,54.545455
2023-10-23,55,34,61.818182
2023-10-30,74,51,68.918919
2023-11-06,107,49,45.794393
2023-11-13,123,64,52.032520
2023-11-20,112,60,53.571429
2023-11-27,82,34,41.463415
2023-12-04,75,36,48.000000


In [44]:
  # Find the day with the highest win ratio
highest_ratio_day = daily_stats['win_ratio'].idxmax()
  # Return the day name 
highest_ratio_day.strftime('%A')

'Monday'

In [45]:
def highest_win_ratio_day(df):
  """
  Finds the day of the week with the highest winning bets ratio.

  Args:
    df: Pandas DataFrame containing betting data with "Date" and "Status" columns.

  Returns:
    String representing the day of the week with the highest win ratio.
  """
  df = df.set_index('Date')
  # Group by day of week and calculate win ratio
  daily_stats = df_copy.groupby(pd.Grouper(level='Date', freq='W-MON')).agg(
      bets=('Status', 'count'),
      wins=('Status', lambda x: x.eq('win').sum())  # Count "win" occurrences
  )
  daily_stats['win_ratio'] = (daily_stats['wins'] / daily_stats['bets']) * 100

  # Find the day with the highest win ratio
  highest_ratio_day = daily_stats['win_ratio'].idxmax()

  # Return the day name
  return highest_ratio_day.strftime('%A')


In [46]:
highest_win_day = highest_win_ratio_day(df.copy())

print(f"Day of week with highest win ratio: {highest_win_day}")

Day of week with highest win ratio: Monday


In [ ]:
# Filter out rows with invalid dates
df = df.dropna(subset=['Date'])

# Add a new column 'Day of Week'
df['Day of Week'] = df['date'].dt.dayofweek

# Filter the DataFrame for wins
wins_df = df[df['Status'] == 'win']

# Group by 'Day of Week' and count wins
wins_per_day = wins_df.groupby('Day of Week').size()

# Calculate average wins for each day (you might need total days per each day of the week for this)
total_days = df['Day of Week'].value_counts()

average_wins_per_day = wins_per_day / total_days

# Find the day with the highest average number of wins
day_with_most_average_wins = int(average_wins_per_day.idxmax())  # or average_wins_per_day.idxmax()

# Convert the day index to a readable string if necessary, e.g., using calendar.day_name


In [54]:
df.groupby('Day of Week').size()

Day of Week
0.0    99
1.0    40
2.0    66
3.0    47
4.0    24
5.0    70
6.0    47
dtype: int64

In [52]:
import calendar
day_name = calendar.day_name[day_with_most_average_wins]

print(f"The day with the highest average number of wins is: {day_name}")

The day with the highest average number of wins is: Wednesday


In [58]:
#find the sub sport if the main sport is Ic 
hockey_df = df[df['Main Sport'] == 'Ic']


In [ ]:
hockey_df.drop('Day', axis=1, inplace=True)

In [ ]:
soccer_df = df[df['Main Sport'] == 'Football']

In [2]:
83+88

171